In [2]:
import subprocess
import os
import time

PYTHON_BIN = "D:/Software/conda/envs/kaggle2/python.exe"

print(f"使用 Python：{PYTHON_BIN}")

# ====== 只运行你指定的脚本 ======
scripts_to_run = [
    # "DM_pipeline_1114原版fix_hkf_lr5e5.py",
    "single_1209.py",
]

for script in scripts_to_run:
    log_file = f"{script}.log"

    print(f"\n后台运行：{script}")
    print(f"日志文件：{log_file}")

    # Windows环境下运行Python脚本
    run_cmd = f"{PYTHON_BIN} {script} > {log_file} 2>&1"

    # 使用 subprocess.Popen 来启动脚本
    subprocess.Popen(
        run_cmd,
        shell=True
    )

    print("等待 20 秒后启动下一个任务...")
    time.sleep(20)

print("\n🎉 选定的训练脚本已在后台启动！")
print("查看运行状态：tasklist | findstr python")


使用 Python：D:/Software/conda/envs/kaggle2/python.exe

后台运行：single_1209.py
日志文件：single_1209.py.log
等待 20 秒后启动下一个任务...

🎉 选定的训练脚本已在后台启动！
查看运行状态：tasklist | findstr python


In [ ]:
import subprocess

target = "D:/Software/conda/envs/kaggle2/python.exe"   # 修改为你的 Python 可执行文件路径

# 获取当前所有 GPU 的 UUID ↔ index 对照表
gpu_map = {}
gpu_query = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,uuid", "--format=csv,noheader,nounits"],
    capture_output=True, text=True
)
for line in gpu_query.stdout.strip().split("\n"):
    if not line.strip():
        continue
    index, uuid = [x.strip() for x in line.split(",")]
    gpu_map[uuid] = index

# 查询当前所有计算进程
result = subprocess.run(
    ["nvidia-smi", "--query-compute-apps=pid,gpu_uuid,gpu_name,used_memory", "--format=csv,noheader,nounits"],
    capture_output=True, text=True
)

print(f"{'List':<5} {'GPU':<5} {'PID':<10} {'Mem(MB)':<10} COMMAND")
print("-" * 80)
count = 0
for list, line in enumerate(result.stdout.strip().split("\n"), 1):
    if not line.strip():
        continue
    pid, gpu_uuid, gpu_name, mem = [x.strip() for x in line.split(",")]
    gpu_index = gpu_map.get(gpu_uuid, "?")
    
    # 在 Windows 上使用 tasklist 查询进程命令
    try:
        # 使用 gbk 编码解码输出，解决 UnicodeDecodeError
        cmd = subprocess.check_output(f'tasklist /FI "PID eq {pid}" /FO LIST', shell=True).decode('gbk')
        if target in cmd:
            # 在 Windows 中，tasklist 命令输出是分行的，所以我们需要提取实际的命令行
            command_line = [line for line in cmd.split('\n') if line.strip().startswith('Image Name')][0]
            print(f"{count:<5} {gpu_index:<5} {pid:<10} {mem:<10} {command_line}")
            count += 1
    except subprocess.CalledProcessError:
        continue

List  GPU   PID        Mem(MB)    COMMAND
--------------------------------------------------------------------------------


In [69]:
subprocess.run(
    ["kill", "196959"]
)

CompletedProcess(args=['kill', '196959'], returncode=0)

In [9]:
import os
import subprocess

def boost_cpu_weight(weight: int = 10000):
    pid = 105230
    try:
        # 找到当前进程的 cgroup 路径
        cgroup_rel = ""
        with open(f"/proc/{pid}/cgroup") as f:
            for line in f:
                if line.startswith("0::"):
                    cgroup_rel = line.strip().split("0::")[1]
                    break

        cgroup_path = f"/sys/fs/cgroup{cgroup_rel}"
        cpu_weight_file = os.path.join(cgroup_path, "cpu.weight")

        if not os.path.exists(cpu_weight_file):
            print("⚠️ 未找到 cpu.weight 文件（可能不是 cgroup v2 系统）。")
            return

        # 提升 CPU 权重
        subprocess.run(["sudo", "bash", "-c", f"echo {weight} > {cpu_weight_file}"], check=True)
        print(f"✅ 已将进程 {pid} 的 CPU 权重提升至 {weight}")

        # 验证结果
        with open(cpu_weight_file) as f:
            print(f"[INFO] 当前 CPU 权重为: {f.read().strip()}")

    except Exception as e:
        print(f"❌ 提升 CPU 权重失败: {e}")

if __name__ == "__main__":
    boost_cpu_weight(10000)


⚠️ 未找到 cpu.weight 文件（可能不是 cgroup v2 系统）。
